# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = pd.read_csv('cities.csv', index_col=0)
df = pd.DataFrame(data_file)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Portland,45.5234,-122.6762,57.61,82,96,4.00,US,1651362828
1,Ushuaia,-54.8000,-68.3000,38.86,70,20,4.61,AR,1651363063
2,Araruama,-22.8728,-42.3431,71.80,92,99,5.73,BR,1651363064
3,Airai,-8.9266,125.4092,69.26,62,69,4.23,TL,1651363065
4,Dingli,35.8614,14.3822,59.81,67,0,9.22,MT,1651363065
...,...,...,...,...,...,...,...,...,...
538,Moerai,-22.4333,-151.3333,78.30,67,47,8.19,PF,1651363402
539,Naze,28.3667,129.4833,63.30,73,100,14.90,JP,1651363402
540,Nadym,65.5333,72.5167,23.74,97,100,7.74,RU,1651363403
541,Nioki,-2.7167,17.6833,71.29,90,34,1.14,CD,1651363404


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [7]:
df['Humidity'].max()

100

In [8]:
# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)
humi = df["Humidity"].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humi, dissipating=False, max_intensity=df['Humidity'].max())
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
vac_spot_df = df[(df['Humidity'] < 80) & (df['Humidity'] > 70) & (df['Wind Speed'] < 10) & (df['Cloudiness'] == 0)]
vac_spot_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,Mar del Plata,-38.0023,-57.5575,50.88,71,0,5.01,AR,1651363083
141,Morondava,-20.2833,44.2833,71.15,78,0,9.17,MG,1651363148
239,Randolph,35.7001,-79.8164,64.45,73,0,4.61,US,1651363213
421,Tazmalt,36.3868,4.3989,50.90,78,0,3.47,DZ,1651363328
537,Aras,42.5617,-2.3560,50.41,71,0,5.75,ES,1651363401


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = vac_spot_df.loc[:, ['City', 'Lat', 'Lng', 'Country']]
hotel_df['Hotel Name'] = ''
hotel_df

,City,Lat,Lng,Country,Hotel Name
32,Mar del Plata,-38.0023,-57.5575,AR,
141,Morondava,-20.2833,44.2833,MG,
239,Randolph,35.7001,-79.8164,US,
421,Tazmalt,36.3868,4.3989,DZ,
537,Aras,42.5617,-2.3560,ES,


In [10]:
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

for index, row in hotel_df.iterrows():
    lati = row['Lat']
    long = row['Lng']
    param = {
        'location' : f'{lati},{long}',
        'radius' : 5000,
        'types' : 'lodging',
        'key' : g_key
            }

    hotel = requests.get(base_url, params=param).json()
    hotel_df.loc[index, 'Hotel Name'] = hotel['results'][0]['name']
    
hotel_df

,City,Lat,Lng,Country,Hotel Name
32,Mar del Plata,-38.0023,-57.5575,AR,Gran Hotel Mar del Plata
141,Morondava,-20.2833,44.2833,MG,La Case Bambou
239,Randolph,35.7001,-79.8164,US,Hampton Inn Asheboro
421,Tazmalt,36.3868,4.3989,DZ,Hôtel Numédia
537,Aras,42.5617,-2.3560,ES,Jardines de Viana Spa


In [12]:
test = hotel
test['results'][0]['name']

'Jardines de Viana Spa'

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))